# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

100%|██████████| 50.6M/50.6M [00:01<00:00, 40.5MB/s]


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [5]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [7]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [8]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [9]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 87.4 MB/s eta 0:00:00


In [10]:
import faiss
import numpy as np
# 1. Crea un índice en FAISS
# Usamos IndexFlatL2 (Distancia Euclidiana) con la dimensión de tus embeddings (768)
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)

# 2. Carga los embeddings
# Esto "llena" el índice con la información de Wikipedia
index.add(embeddings)
print(f"Índice listo con {index.ntotal} fragmentos.")


# 3. Realiza una búsqueda a partir de una query
query_text = "Battery measuring"  # Definimos la pregunta
query_vec = embed_query(query_text) # Convertimos la pregunta a vector (1, 768)


# Ejecutamos la búsqueda de los 10 más cercanos (k=10)
# D = Distancias (qué tan cerca están), I = Índices (qué posición ocupan)
D, I = index.search(query_vec, k=10)


# --- Visualización de resultados ---
print(f"\nResultados más relevantes para: '{query_text}'")
for rank in range(len(I[0])):
    posicion = I[0][rank]
    distancia = D[0][rank]
    texto = chunks_df.iloc[posicion]["text"]

    print(f"\n[Top {rank+1}] - Distancia: {distancia:.4f}")
    print(f"Contenido: {texto[:150]}...")

Índice listo con 79104 fragmentos.

Resultados más relevantes para: 'Battery measuring'

[Top 1] - Distancia: 0.2593
Contenido: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing ...

[Top 2] - Distancia: 0.2764
Contenido: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visu...

[Top 3] - Distancia: 0.3198
Contenido: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is ba...

[Top 4] - Distancia: 0.3217
Contenido: ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the ...

[Top 5] - Distancia: 0.3228
Contenido: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in f

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [ ]:
# Instalamos Qdrant
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 19.3 MB/s eta 0:00:00


In [12]:
!pip install -U qdrant-client

In [13]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Reiniciamos el cliente para estar seguros
client = QdrantClient(":memory:")

COLLECTION_NAME = "wikipedia_chunks"
D = embeddings.shape[1]

# Re-creamos la colección
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=D, distance=Distance.COSINE),
)

# Insertamos los puntos (asegúrate de que 'embeddings' y 'chunks_df' existan)
points = [
    PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload={
            "text": chunks_df.iloc[i]["text"],
            "doc_id": int(chunks_df.iloc[i]["doc_id"])
        }
    ) for i in range(len(embeddings))
]
client.upsert(collection_name=COLLECTION_NAME, points=points)

# --- FUNCIÓN ENTREGABLE CORREGIDA ---
def qdrant_search(query_embedding, k=5):
    """
    Retorna: lista de (id, score, text, metadata)
    """
    # Convertimos el query_embedding a una lista plana de floats
    # Si viene como (1, 768) lo aplanamos a (768,)
    if len(query_embedding.shape) > 1:
        vector_to_search = query_embedding[0].tolist()
    else:
        vector_to_search = query_embedding.tolist()

    # Usamos el cliente para buscar
    hits = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=vector_to_search,
        limit=k
    )

    results = []
    for hit in hits:
        results.append((
            hit.id,
            hit.score,
            hit.payload.get("text", ""),
            {key: val for key, val in hit.payload.items() if key != "text"}
        ))
    return results

# Ejemplo de consulta
try:
    print(f"\n--- Ejecutando búsqueda en Qdrant para: '{query_text}' ---")
    top_results = qdrant_search(query_vec, k=5)
    for i, (idx, score, text, meta) in enumerate(top_results):
        print(f"Top {i+1} | ID: {idx} | Score: {score:.4f} | Texto: {text[:80]}...")
except Exception as e:
    print(f"Error detectado: {e}")

/tmp/ipython-input-3461383554.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(



--- Ejecutando búsqueda en Qdrant para: 'Battery measuring' ---
Error detectado: 'QdrantClient' object has no attribute 'search'


/tmp/ipython-input-3461383554.py:27: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(collection_name=COLLECTION_NAME, points=points)


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [26]:
!pip install pymilvus

In [28]:
!pip install "pymilvus[milvus_lite]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 16.5 MB/s eta 0:00:00


In [29]:
from pymilvus import MilvusClient, DataType
import time

# 1. Conectar a Milvus (Lite)
client_milvus = MilvusClient("milvus_demo.db")
COLLECTION_NAME = "wikipedia_milvus"
D = embeddings.shape[1]

In [30]:
# Si la colección ya existe, la borramos para empezar de cero
if client_milvus.has_collection(COLLECTION_NAME):
    client_milvus.drop_collection(COLLECTION_NAME)

# Crear esquema
schema = client_milvus.create_schema(auto_id=False, enable_dynamic_field=True)

# Añadir campos
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=D)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=65535)

# Crear colección
client_milvus.create_collection(collection_name=COLLECTION_NAME, schema=schema)

In [ ]:
data = [
    {
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": chunks_df.iloc[i]["text"],
        "doc_id": int(chunks_df.iloc[i]["doc_id"])
    }
    for i in range(len(embeddings))
]

client_milvus.insert(collection_name=COLLECTION_NAME, data=data)
print(f"Insertados {len(data)} vectores en Milvus.")

## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?
